In [1]:
import pandas as pd
import plotly.graph_objects as go

from bson.code import Code
from pymongo import MongoClient

In [2]:
client = MongoClient("localhost:27017")
db = client.hotels

In [3]:
set_map = Code("""
function(){
    var key = {name: this.name, lng: this.lng, lat: this.lat}
    var value = {
        the_id: this.name,
        count: 1,
        positive: this.positive
    }

    emit(key, value)
}
""")

set_reduce = Code("""
function(key, values){
    reducedObj = {
        the_id: key,
        count: 0,
        positive: 0,
    }

    values.forEach( function(value) {
        reducedObj.count += value.count;
        reducedObj.positive += value.positive;
    });
    
    return reducedObj
}
""")

set_finalize = Code("""
function(key, values){
    fin_obj = {
        name: key.name,
        lat: key.lat,
        lng: key.lng,
        reviews: values.count,
        positive: values.positive
    }
    return fin_obj
}
""")

result = db.reviews.map_reduce(set_map, set_reduce, "results", finalize=set_finalize)

KeyboardInterrupt: 

In [ ]:
find_result = result.find({}, { '_id': 0, 'value': 1})
result_list = [x['value'] for x in find_result]
result_df = pd.DataFrame(result_list)

In [ ]:
print(len(list(result_df['name'])))
print(len(list(result_df['lat'])))
print(len(list(result_df['lng'])))

In [ ]:
ACCESS_TOKEN = 'pk.eyJ1Ijoic2ltb25odmEiLCJhIjoiY2s1OXk2c3BhMTFyaDNwbGprYWowZmZhOSJ9.Alw4kvUDHu7cxG9hB_Ra9Q'

fig = go.Figure(
    go.Scattermapbox(
        lat=list(result_df['lat']),
        lon=list(result_df['lng']),
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=9
        ),
        text=list(result_df['name'])
    )
)

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=ACCESS_TOKEN,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=48.203745,
            lon=16.335677
        ),
        pitch=0,
        zoom=4
    )
)
fig.show()